<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深度學習(Deep learning)
原本指的是「深度」神經網路，線變成簡單神經網路。

# 張量

# 層的抽象概念

# 線性層

# 把神經網路視為一系列的層

# 損失與最佳化

# 範例：XOR再次嘗試

# 其他激活函數

# **範例**：FizzBuzz 再次嘗試

# Softmax與交叉熵

# Deopout隨機拋棄

# 範例：MNIST

# 模型的儲存與載入